# An email agent that goes through my inbox

1. Authenticates the user
2. Reads an email
3. Sends emails with human approval required

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# setting dataclass with Email structure
from dataclasses import dataclass
from langchain.agents import AgentState

@dataclass
class EmailContext:
    email_address: str = "sajid.ahmed123@gmail.com"
    password: str = "sajid.ahmed"

# boolean True/False if the user is authenticated
class AuthenticatedState(AgentState):
    authenticated: bool

In [3]:
from langchain.tools import tool, ToolRuntime
from langchain.messages import ToolMessage
from langgraph.types import Command

@tool
def check_inbox() -> str:
    """Check the inbox for recent emails"""
    return """
    Hi Julie, 
    I'm going to be in town next week and was wondering if we could grab a coffee?
    - best, Jane (jane@example.com)
    """

@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Send an email to the given address with the given subject and body."""
    # fake email sending
    return f"Email sent to {to} with subject {subject} and body {body}"

@tool
def authenticate(email: str, password: str, runtime: ToolRuntime) -> Command:
    """Authenticate the user with the given email and password"""
    if email == runtime.context.email_address and password == runtime.context.password:
        return Command(
            update={
                "authenticated": True,
                "messages": [
                    ToolMessage("Successfully authenticated", tool_call_id=runtime.tool_call_id)
                ],
            }
        )
    else:
        return Command(
            update={
                "authenticated": False,
                "messages": [
                    ToolMessage("Authentication failed", tool_call_id=runtime.tool_call_id)
                ],
            }
        )

In [4]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable

@wrap_model_call
def dynamic_tool_call(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:

    """Dynamically call tools based on the runtime context"""

    authenticated_user = request.state.get("authenticated")
    
    if authenticated_user == True:
        tools = [check_inbox,send_email]
    else:
        tools = [authenticate]

    request = request.override(tools=tools)
    return handler(request)

In [5]:
from langchain.agents.middleware import dynamic_prompt
authenticated_prompt = "You are a helpful assistant that can check the inbox and send emails."
unauthenticated_prompt = "You are a helpful assistant that can authenticate users."


@dynamic_prompt
def dynamic_prompt_func(request: ModelRequest) -> str:
    """Generate system prompt based on authentication status"""
    authenticated = request.state.get("authenticated")

    if authenticated:
        return authenticated_prompt
    else:
        return unauthenticated_prompt

In [6]:
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langchain.agents import create_agent

agent = create_agent(
        "gpt-5-nano",
        tools=[authenticate, check_inbox, send_email],
        state_schema=AuthenticatedState,
        context_schema=EmailContext,
        middleware=[
            dynamic_tool_call,
            dynamic_prompt_func,
            HumanInTheLoopMiddleware(
                interrupt_on={
                    "authenticate": False,
                    "check_inbox": False,
                    "send_email": True,
                }
            ),
        ],
    )


## Testing

In [11]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "2"}}

response = agent.invoke(
    {"messages": [HumanMessage(content="Check my inbox for any new messages")]},
    context=EmailContext(),
    config=config
)

print(response['messages'][-1].content)

I can’t directly access your inbox from here. I don’t have a built-in mail client to fetch messages.

Options:
- If you want, I can help you sign in to your email account so you can review new messages yourself. If you’d like to proceed, provide:
  - Your email address
  - Your password
  - Which provider you use (Gmail, Outlook, Yahoo, etc.)

- If you’d rather not share credentials, I can guide you through checking your inbox manually or on your device (step-by-step for Gmail, Outlook, etc.), and I can help you search for unread messages (e.g., Gmail: is:unread).

Tell me how you’d like to proceed.


In [13]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="sajid.ahmed123@gmail.com,sajid.ahmed")]},
    context=EmailContext(),
    config=config
)

print(response['messages'][-1].content)

/Users/sajidahmed/Desktop/VS Code/lca-lc-foundations/.venv/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='context', input_value=EmailContext(email_addres... password='sajid.ahmed'), input_type=EmailContext])
  return self.__pydantic_serializer__.to_python(


I found 1 new message in your inbox:

From: Jane (jane@example.com)
Subject: (no subject)
Message: "Hi Julie, I'm going to be in town next week and was wondering if we could grab a coffee? - best, Jane (jane@example.com)"

What would you like to do?
- Reply to Jane (I can draft a reply for you)
- Archive or delete
- Mark as read or unread

If you’d like, here are two draft replies you can choose from or customize:

Draft 1 (friendly):
Hi Jane, great to hear from you! I'd love to catch up. I'm around next week—are you free Tuesday or Thursday afternoon? If another time works better, let me know. Looking forward to it!

Draft 2 (slightly more formal):
Hi Jane, thanks for reaching out. I’d be happy to meet for coffee while you’re in town. Could you share a couple of times that work next week? Best regards, [Your Name]

Tell me which draft you prefer or provide details (date/time/place) and I’ll send it.


In [14]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="Draft a response to Jane")]},
    context=EmailContext(),
    config=config
)

print(response['messages'][-1].content)

I’d be happy to draft it, but I’m missing a bit of context. Could you tell me:
- What’s the purpose of the message (e.g., reply to an email, text, invitation, follow-up)?
- What’s your relationship with Jane (coworker, friend, client, etc.)?
- What are the key points you want to include?
- What tone would you like (professional, friendly, concise)?
- Any deadlines or actions you want to propose?

In the meantime, here are a couple of ready-to-fill templates you can use. Just replace the placeholders.

1) Professional email
Subject: Re: [Topic]
Hi Jane,
Thank you for the update on [topic]. I’ve reviewed the details and agree with the proposed plan. I can [your action]. Could we schedule a quick call to confirm timing? I’m available at [times/days]. Please let me know what works for you.
Best regards,
[Your Name]

2) Friendly text
Hey Jane! Thanks for the note. Sounds good to me. Do you want to chat about the details, or should I start on [task]? I’m free [times]. Let me know what works!

In [15]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="Draft a response to Jane in a friendly text")]},
    context=EmailContext(),
    config=config
)

print(response['messages'][-1].content)

I can draft several friendly options. Could you share a bit more about the situation or paste Jane’s message? In the meantime, here are a few quick templates you can use or tweak:

1) Casual plan to meet up
Hey Jane! Great to hear from you. I’d love to catch up. Are you free this weekend? I can do Saturday afternoon or Sunday morning—let me know what works. 😊

2) Quick check-in
Hi Jane! Hope you’re doing well. How’s everything going with you? Would love to hear what you’ve been up to.

3) Grateful or appreciative
Hey Jane, thanks so much for [X]. That was really thoughtful of you. Let’s chat soon—coffee on me next time!

4) After a delayed reply
Hey Jane! Sorry for the slow reply—things have been hectic. How have you been? Want to grab coffee this week to catch up?

5) If Jane asked for help
Hi Jane! Happy to help. Could you share a bit more detail on [topic]? I can get started and loop you in.

If you share the context or Jane’s exact message, I’ll tailor a single polished reply that 

In [16]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="Quick check-in")]},
    context=EmailContext(),
    config=config
)

print(response['messages'][-1].content)

Nice to hear from you! How can I help today? Here are a couple of quick options:

- Sign in: provide your email and password and I’ll attempt to authenticate you.
- Reset password: I can guide you through the steps or help with a reset link.
- Check status: if you already have a session and want to verify something, tell me what you need.

What would you like to do? If you choose to sign in, please share your email and password. I’ll handle the rest. (I’ll keep your credentials private and only use them to authenticate.)


In [17]:
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

KeyError: '__interrupt__'